In [15]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

In [10]:
ticker = input("Enter Ticker")
print(ticker)

AAPL


In [11]:
from finvizfinance.news import News
fnews = News()
all_news = fnews.get_news()
all_news

{'news':        Date                                              Title  \
 0   06:24PM  Impossible Foods CEO chews out Bloomberg artic...   
 1   06:24PM  President of St. Louis Fed pushes for steeper ...   
 2   06:08PM  Stocks moving in after-hours: Nvidia, Lucid, E...   
 3   06:06PM  Subway adds EV charging parks in competition w...   
 4   06:06PM    JPMorgan restricts employees from using ChatGPT   
 ..      ...                                                ...   
 85  07:38AM         What's behind the fruit and veg shortages?   
 86  07:32AM  Futures stable after Wall St rout on rate worries   
 87  07:23AM  China’s Economic Support for Russia Could Elic...   
 88  07:17AM      S&P 500 index seen climbing 5% by end of 2023   
 89  07:14AM  Diversity, equity and inclusion jobs slashed a...   
 
                Source                                               Link  
 0     foxbusiness.com  https://foxbusiness.com/economy/impossible-foo...  
 1     foxbusiness.com  https://fo

In [12]:
# Get Data
n = 3
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

#for ticker in tickers:
url = finwiz_url + ticker
print("current url is: " +url)
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36."}
req = Request(url=url,headers=header) 
resp = urlopen(req)    
html = BeautifulSoup(resp, features="lxml")
news_table = html.find(id='news-table')
news_tables[ticker] = news_table

try:
    #for ticker in tickers:
    df = news_tables[ticker]
    df_tr = df.findAll('tr')
    
    print ('\n')
    print ('Recent News Headlines for {}: '.format(ticker))
        
    for i, table_row in enumerate(df_tr):
        a_text = table_row.a.text
        td_text = table_row.td.text
        td_text = td_text.strip()
        print(a_text,'(',td_text,')')
        if i == n-1:
            break
except KeyError:
    pass

current url is: https://finviz.com/quote.ashx?t=AAPL


Recent News Headlines for AAPL: 
Why Abbott Laboratories Stock Sank Today ( Feb-22-23 06:07PM )
President Biden upholds ITC ruling on Apple Watchs EKG patent battle ( 04:07PM )
Dexcom Slumps As A New Rival Reportedly Arrives On Scene: Apple ( 04:04PM )


In [13]:
# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        #print(x.get_text())
        text = x.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text ])
print(parsed_news)

[['AAPL', 'Feb-22-23', '06:07PM', 'Feb-22-23 06:07PMWhy Abbott Laboratories Stock Sank Today Motley Fool'], ['AAPL', 'Feb-22-23', '04:07PM', '04:07PMPresident Biden upholds ITC ruling on Apple Watchs EKG patent battle Yahoo Finance Video'], ['AAPL', 'Feb-22-23', '04:04PM', "04:04PMDexcom Slumps As A New Rival Reportedly Arrives On Scene: Apple Investor's Business Daily"], ['AAPL', 'Feb-22-23', '03:33PM', '03:33PMMicrosoft is bringing ChatGPT-powered Bing to your smartphone, challenging Google Yahoo Finance'], ['AAPL', 'Feb-22-23', '02:31PM', "02:31PMSpruill's hot take: Incentivize Triangle startups  not Apple, Google and Amazon American City Business Journals"], ['AAPL', 'Feb-22-23', '12:53PM', '\n12:53PM\nLoading…\n'], ['AAPL', 'Feb-22-23', '12:53PM', '12:53PMApple Makes Major Progress on No-Prick Blood Glucose Tracking for Its Watch Bloomberg'], ['AAPL', 'Feb-22-23', '11:22AM', '11:22AMMicrosoft Brings ChatGPT-Powered Bing to Apple, Android Mobile Devices The Wall Street Journal'], [

In [16]:
df = pd.DataFrame(parsed_news, columns =['Ticker', 'Date', 'Time',"Headline"])

In [18]:
print(df.head)

<bound method NDFrame.head of     Ticker       Date     Time  \
0     AAPL  Feb-22-23  06:07PM   
1     AAPL  Feb-22-23  04:07PM   
2     AAPL  Feb-22-23  04:04PM   
3     AAPL  Feb-22-23  03:33PM   
4     AAPL  Feb-22-23  02:31PM   
..     ...        ...      ...   
98    AAPL  Feb-15-23  01:39AM   
99    AAPL  Feb-15-23  12:27AM   
100   AAPL  Feb-14-23  07:35PM   
101   AAPL  Feb-14-23  05:55PM   
102   AAPL  Feb-14-23  05:52PM   

                                              Headline  
0    Feb-22-23 06:07PMWhy Abbott Laboratories Stock...  
1    04:07PMPresident Biden upholds ITC ruling on A...  
2    04:04PMDexcom Slumps As A New Rival Reportedly...  
3    03:33PMMicrosoft is bringing ChatGPT-powered B...  
4    02:31PMSpruill's hot take: Incentivize Triangl...  
..                                                 ...  
98   01:39AMFedspeak, U.S. retail sales, U.K. infla...  
99   12:27AMEuropes Big Tech trust buster Financial...  
100  Feb-14-23 07:35PMApple to Scrutinize Custom

In [20]:
filename = "datasets/" + ticker + "-recent.csv"
df.to_csv(filename)